# Cython

### Adrian M. Price-Whelan

C-like speed with Python-like code.

Cython compiles Python down to C, but not in the way you think. It tries to be smart about how it does this in order to get you the best performance. We're going to do a few simple demos in the notebook using the `cythonmagic` IPython magic extension, but then we'll dive in to see what is actually happening (what the 'magic' is), and how you can write scripts and modules with Cython.

A few quick warnings:
- It's usually fairly easy to get 2x speed increases (by changing a few lines of code), but getting huge speed boosts (factors of 10-100) may require rethinking your code and re-writing things to look more like C.
- The documentation is terrible (if I had a nickel...).
- Things get complicated pretty fast, so don't dive in unless you really need to optimize!

---

Some imports we'll need later...

In [92]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import integrate

## Case study: 

Let's imagine we are developing some analysis code. As part of a function that is called many, many times (e.g., inside our MCMC chain), we need to do some numerical integration. The function we need to integrate is complicated, e.g., a functional expansion of the form:

In [93]:
def f_py(r):
    r = np.atleast_1d(r)
    rho = np.zeros(len(r))
    
    xsi = (r-1.)/(r+1.)

    C = np.zeros((128,len(r)))
    C[0] = 1.0
    C[1] = 3.*xsi
    for n in range(2, 128):
        C[n] = (1./n) * (2*xsi*(n+0.5)*C[n-1] - (n+1.)*C[n-2])

    for n in range(128):
        K_n = 0.5*n*(n+3.)
        rho += K_n/(2*np.pi) / (r * (1+r)**3) * C[n]

    return rho

We can, of course, use the integration tools from Scipy to do this integration. Let's say the grid we need to evaluate the domain we need to integrate over is
$$
x \in (0.1, 4.)
$$

In [94]:
x = np.linspace(0.1, 4., 16384)

In [99]:
y = f_py(x)
integrate.simps(y, x=x)

274.15417518426352

Well, that seemed to be pretty fast...but if this is inside of an MCMC likelihood call, it can add up! Looking back at the function above, it's pretty obvious why it is slow: we need to compute some coefficients that depend recursively on the elements before it, so the code currently uses loops to do this. Let's see how slow it actually is to evaluate the function on the above grid and do the integration:

In [100]:
%%timeit
y = f_py(x)
integrate.simps(y, x=x)

10 loops, best of 3: 74.9 ms per loop


Brutal! 80 ms? We can speed this up...

In [101]:
%%cython
import numpy as np
cimport numpy as np

cpdef f_cy1(double[::1] r): # we tell Cython to expect a 1D array of doubles (floats)
    cdef:
        int i, n
        int nelems = r.shape[0]
        double K_n
        double[::1] rho = np.zeros(nelems)
        double[::1] xsi = np.zeros(nelems)
        double[:,::1] C = np.zeros((128,nelems))
    
    for i in range(nelems):
        xsi[i] = (r[i]-1.) / (r[i]+1.)
    
    for i in range(nelems):
        C[0,i] = 1.0
        C[1,i] = 3.*xsi[i]
        for n in range(2, 128):
            C[n,i] = (1./n) * (2*xsi[i]*(n+0.5)*C[n-1,i] - (n+1.)*C[n-2,i])

    for n in range(128):
        K_n = 0.5*n*(n+3.)
        for i in range(nelems):
            rho[i] += K_n/(2*np.pi) / (r[i] * (1+r[i])**3) * C[n,i]

    return rho

In [102]:
%%timeit
y = f_cy1(x)
integrate.simps(y, x=x)

1 loops, best of 3: 381 ms per loop


\>300 ms?? We made it __slower__! What happened?

Let's use `cython -a` to see what's going on...

In [103]:
%%cython -a
import numpy as np
cimport numpy as np

cpdef f_cy1(double[::1] r): # we tell Cython to expect a 1D array of doubles (floats)
    cdef:
        int i, n
        int nelems = r.shape[0]
        double K_n
        double[::1] rho = np.zeros(nelems)
        double[::1] xsi = np.zeros(nelems)
        double[:,::1] C = np.zeros((128,nelems))
    
    for i in range(nelems):
        xsi[i] = (r[i]-1.) / (r[i]+1.)
    
    for i in range(nelems):
        C[0,i] = 1.0
        C[1,i] = 3.*xsi[i]
        for n in range(2, 128):
            C[n,i] = (1./n) * (2*xsi[i]*(n+0.5)*C[n-1,i] - (n+1.)*C[n-2,i])

    for n in range(128):
        K_n = 0.5*n*(n+3.)
        for i in range(nelems):
            rho[i] += K_n/(2*np.pi) / (r[i] * (1+r[i])*(1+r[i])*(1+r[i])) * C[n,i]

    return rho

Aha, so the loops are very slow because Python is still worrying about things like checking the boundaries of the array. Also, `np.pi` is triggering some kind of module lookup

In [104]:
%%cython -a
import numpy as np
cimport numpy as np
import cython

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.nonecheck(False)
cpdef f_cy2(double[::1] r): # we tell Cython to expect a 1D array of doubles (floats)
    cdef:
        int i, n
        int nelems = r.shape[0]
        double K_n, pi = np.pi
        double[::1] rho = np.zeros(nelems)
        double[::1] xsi = np.zeros(nelems)
        double[:,::1] C = np.zeros((128,nelems))
    
    for i in range(nelems):
        xsi[i] = (r[i]-1.) / (r[i]+1.)
    
    for i in range(nelems):
        C[0,i] = 1.0
        C[1,i] = 3.*xsi[i]
        for n in range(2, 128):
            C[n,i] = (1./n) * (2*xsi[i]*(n+0.5)*C[n-1,i] - (n+1.)*C[n-2,i])

    for n in range(128):
        K_n = 0.5*n*(n+3.)
        for i in range(nelems):
            rho[i] += K_n/(2*pi) / (r[i] * (1+r[i])*(1+r[i])*(1+r[i])) * C[n,i]

    return rho

In [105]:
%%timeit
y = f_cy2(x)
integrate.simps(y, x=x)

10 loops, best of 3: 47.8 ms per loop


Ok, there's a speed boost!